# Week 8 - 27th September, 2023

1. Build a logistic regression model to predict the probability of getting `chd` (Coronary Heart Disease), that is, P(`chd` = 1). Use all other columns except chd as features to build the model.
2. Find out which features are statistically significant in the logistic regression model built in Question 1. Build a new logistic regression model using only the significant features.
3. From the logistic regression model parameters, determine which parameters affect the probability of `chd` positively and negatively.
4. Calculate Youden’s index for all possible cut-off probabilities ranging from `0.1` to `0.5` with an interval of `0.01`. Find the optimal cut-off where Youden’s index is maximum.
5. Build a confusion matrix based on the cut-off probability found in Question 4 and report the precision and recall of the model for chd cases (i.e., `chd` = 1).
6. Find optimal cut-off probability using cost-based approach using the cost of FPs and FNs as defined below:
   - Cost of predicting “chd” as “No chd” (FNs) cases is 5 times more than predicting “No chd” as “chd” (FPs).
   - Then find the precision and recall of the model for `chd` = 1.

In [1]:
import pandas as pd

input_df = pd.read_csv('../classification-datasets/HeartEW.csv')
input_df.head(5)

,Class,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13
0,1,34,1,1,118,182,0,2,174,0,0.0,1,0,3
1,2,53,1,4,140,203,1,2,155,1,3.1,3,0,7
2,2,61,1,1,134,234,0,0,145,0,2.6,2,2,3
3,1,68,1,3,118,277,0,0,151,0,1.0,1,1,7
4,2,54,1,4,122,286,0,2,116,1,3.2,2,2,3
